# Computatially Modelling a Reaction

## Imports
Finds all needed packages and if not present installs them via pip.
Somewhat sloppy since this will also force conda environments to use pip to install packages available via conda.

In [ ]:
try:
    from pyscf import gto, dft
    print("Found pyscf")
except:
    print("Can't import, installing via pip")
    !pip3 install pyscf
    from pyscf import gto

try:
    import numpy as np
    print("Found numpy")
except:
    print("Can't import, installing via pip")
    !pip3 install numpy
    import numpy as np

try:
    import matplotlib.pyplot as plt
    print("Found matplotlib")
except:
    print("Can't import, installing via pip")
    !pip3 install matplotlib
    import matplotlib.pyplot as plt

try:
    from ase import Atoms
    from ase.build import molecule
    from ase.visualize import view
    from ase.io import write
    print("Found ase")
except:
    print("Can't import, installing via pip")
    !pip3 install ase
    from ase import Atoms    
    from ase.build import molecule
    from ase.visualize import view
    from ase.io import write

try:
    from pyscf.geomopt.geometric_solver import optimize
    print("Found geometry module")
except:
    print("Can't import, installing via pip")
    !pip3 install "pyscf[geomopt]"
    from pyscf.geomopt.geometric_solver import optimize

## Build Molecular Structure

The molecular structure was created via avogadro, which offers an output to .txt files.
It needs to be optimised, since Avogrado is not a simulation package and only offers "sketches".

In [ ]:
moleculename = 'sn2'

mol = gto.Mole()
mol.build(atom = """
F  -1.860898   0.218170   0.003117
C  -0.393486   0.210786  -0.001531
H   2.244246   0.198753  -0.009416
H  -0.003547  -0.827583  -0.012933
H   0.007238   0.718176   0.899774
H   0.001646   0.735740  -0.895220""",
          basis = 'ccpvdz', 
          spin = 0,
          charge = -1,
          unit = 'angstrom')

#atom = inputatoms,
#atom = 'O 0.0 0.0 0.119262 ; H 0.0 0.763239 -0.477047 ; H 0.0 -0.763239 -0.477047',

Define dft class object, functional and optimisation algorithm

In [ ]:
mf_hf = dft.RKS(mol) # Define dft class object
mf_hf.xc = 'pbe' # default
mf_hf = mf_hf.newton() # second-order algortihm
mf_hf.kernel()

Set convergence parameters for simulation

In [ ]:
conv_params = { # These are the default settings
    'convergence_energy': 1e-6,  # Eh
    'convergence_grms': 3e-4,    # Eh/Bohr
    'convergence_gmax': 4.5e-4,  # Eh/Bohr
    'convergence_drms': 1.2e-3,  # Angstrom
    'convergence_dmax': 1.8e-3,  # Angstrom
}

In [ ]:
dft_energies = []
def cb(envs):
    mf_hf = envs["g_scanner"].base
    dft_energies.append(mf_hf.e_tot)

In [ ]:
# optimise initial structure
mol_eq = optimize(mf_hf, **conv_params, maxsteps=100, callback=cb)

In [ ]:
print("Optimised Structure:")
print(mol_eq.elements)
print(mol_eq.atom_coords() * 0.529177 )

## View Optimised Structure

In [ ]:
optmolecule = Atoms(mol_eq.elements, positions=mol_eq.atom_coords() * 0.529177) # * 0.529177)

In [ ]:
view(optmolecule, viewer='x3d')

In [ ]:
print(dft_energies[-1])
# Writing to a file while converting from Hartree to eV
write(f'{round(dft_energies[-1] * 27.2114079527, 3)} -{moleculename}.xyz', optmolecule)

## Find Initial state or Final state

In [ ]:
optimisedcoordinates = []

for element, coords in zip(mol_eq.elements, mol_eq.atom_coords()):
    optimisedcoordinates.append(f"{element} {' '.join(map(str, coords * 0.529177))}")

optimisedinput = ' ; '.join(optimisedcoordinates)


In [ ]:
# Build Molecule from optimised input
mol = gto.Mole()
mol.build(atom = optimisedinput,
          basis = 'ccpvdz', 
          spin = 0,
          charge = -1,
          unit = 'angstrom')

In [ ]:
mf_hf = dft.RKS(mol)
mf_hf.xc = 'pbe' # default
mf_hf = mf_hf.newton() # second-order algortihm
mf_hf.kernel()

In [ ]:
# This is the only thing different from earlier
# Constraints are telling it to scan certain bond distances and write to xyz file
conv_params = { # These are the default settings
    'convergence_energy': 1e-6,  # Eh
    'convergence_grms': 3e-4,    # Eh/Bohr
    'convergence_gmax': 4.5e-4,  # Eh/Bohr
    'convergence_drms': 1.2e-3,  # Angstrom
    'convergence_dmax': 1.8e-3,  # Angstrom
}
params = {"constraints": "constraints.txt"}

### Run Simulation

WARNING: This takes ~10 minutes

In [ ]:
mol_eq = optimize(mf_hf, **conv_params, **params, maxsteps=500, callback=cb)

### Find in Simulation Run

The results are saved to scan-final.xyz, the small parser below gets energies and configurations for runs.

In [ ]:
# Now go through scan file and find maximum between states and minimum after state
# Optimise FS with top part
# Optimise TS (top) with part below

import re

with open("scan-final.xyz", "r") as f:
    file_content = f.read()


distances = re.findall("Distance 2-3 = ([0-9|\.]*)", file_content)
distances = [float(distance) for distance in distances]
energies = re.findall("Energy -([0-9|\.]*)", file_content)
energies = [-float(energy) for energy in energies]

i_is = np.argmin(energies)
i_emax = np.argmax(energies)
i_fs = np.argmin(energies[:i_emax-1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3))

ax.plot(distances, energies, 'o')
ax.plot(distances[i_is], energies[i_is], "ro", label='IS')
ax.plot(distances[i_emax], energies[i_emax], "go", label='TS')
ax.plot(distances[i_fs], energies[i_fs], "ko", label='FS')
ax.legend()
ax.set_ylabel("Energy [eV]")
ax.set_xlabel("Bond Distance [A]")

ax.set_title("Scan Results")

plt.show()

In [ ]:
split_lines = file_content.split('\n')

counter = 0
atoms = []
while counter<len(split_lines)-1:
    n_atoms = int(split_lines[counter])
    counter += 2
    
    atom = []
    for at_count in range(n_atoms):
        atom.append(split_lines[counter])
        counter += 1

    atoms.append("\n".join(atom))

## Optimise transition state close to scan value

The transition state near the maximum Gibbs Free Energy (G) is not exact since it was only estimated by having the molecules run at certain distances from one another.
Below we optimise the transition state to be at the true $G_{max}$.

In [ ]:
def optimise_state(molecule):
    """Optimise molecule to be at max Gibbs Free Energy.

    Args:
        molecule (gto.Mole): Input configuration.

    Returns:
        gto.Mole: Configuration of maximum Gibbs Free Energy.
    """
    mol = gto.Mole()
    mol.build(
        atom = molecule,
        basis = 'ccpvdz', 
        spin = 0,
        charge = -1,
        unit = 'angstrom'
    )

    mf_hf = dft.RKS(mol)
    mf_hf.xc = 'pbe' # default
    mf_hf = mf_hf.newton() # second-order algortihm
    mf_hf.kernel()

    conv_params = { # These are the default settings
        'convergence_energy': 1e-6,  # Eh
        'convergence_grms': 3e-4,    # Eh/Bohr
        'convergence_gmax': 4.5e-4,  # Eh/Bohr
        'convergence_drms': 1.2e-3,  # Angstrom
        'convergence_dmax': 1.8e-3,  # Angstrom
    }
    params = {'transition': True}
    mol_eq = optimize(mf_hf, **conv_params, **params, maxsteps=500, callback=cb)
    return mol_eq

eq_ts = optimise_state(atoms[i_emax])

Now we look at the console output of the optimisation and copy the final structure into `optimised_ts_atom`.

In [ ]:
# Copy pasted from output
optimised_ts_energy = -140.083029488409
optimised_ts_atom = """
   F  -1.900358   0.425170   0.252872
   C  -0.172879   0.282364   0.086040
   H   1.719134   0.126274  -0.096672
   H  -0.079296  -0.808202   0.056109
   H   0.143874   0.786928   1.004747
   H  -0.032821   0.822831  -0.855881
"""

## Calculate Thermoproperties
This part takes all the structures and gives you Gibbs Free Energies for them.
But be careful, for initial state do Fluorine separate from CH4

In [ ]:
from pyscf.hessian import thermo

In [ ]:
symbols = optmolecule.get_chemical_symbols()
coordinates = optmolecule.get_positions()

init_state_fluorine_molecule = gto.Mole()
init_state_hc_molecule = gto.Mole()

init_state_fluorine_atom = [[symbol, tuple(coords.tolist())] for (symbol, coords) in zip(symbols, coordinates) if symbol == 'F']
init_state_hc_atom = [[symbol, tuple(coords.tolist())] for (symbol, coords) in zip(symbols, coordinates) if symbol != 'F']
init_state_fluorine_molecule = init_state_fluorine_molecule.build(
    atom=init_state_fluorine_atom,
    basis='ccpvdz',
    spin=0,
    charge=-1,
    unit='bohr'
)
init_state_hc_molecule = init_state_hc_molecule.build(
    atom=init_state_hc_atom,
    basis='ccpvdz',
    spin=0,
    charge=0,
    unit='bohr'
)

initial_state_molecule = gto.Mole()
initial_state_molecule.build(atom=atoms[i_is], basis='ccpvdz', spin=0, charge=-1, unit='angstrom')

ts_molecule = gto.Mole()
ts_molecule.build(atom=optimised_ts_atom, basis='ccpvdz', spin=0, charge=-1, unit='angstrom')

fs_molecule = gto.Mole()
fs_molecule.build(atom=atoms[i_fs],  basis='ccpvdz', spin=0, charge=-1, unit='angstrom')

In [ ]:
def get_freq_therm(mol, verbose=False):
    mf_hf = dft.RKS(mol)
    mf_hf.xc = 'pbe' # default
    mf_hf = mf_hf.newton() # second-order algortihm
    mf_hf.kernel()
    hessian = mf_hf.Hessian().kernel()
    freq_info = thermo.harmonic_analysis(mf_hf.mol, hessian)
    # Thermochemistry analysis at 298.15 K and 1 atmospheric pressure
    thermo_info = thermo.thermo(mf_hf, freq_info['freq_au'], 298.15, 101325)
    
    if verbose:
        print('Total electronic energy')
        print(thermo_info['E0' ])

        print('Rotation constant')
        print(thermo_info['rot_const'])

        print('Zero-point energy')
        print(thermo_info['ZPE'   ])

        print('Internal energy at 0 K')
        print(thermo_info['E_0K'  ])

        print('Internal energy at 298.15 K')
        print(thermo_info['E_tot' ])

        print('Enthalpy energy at 298.15 K')
        print(thermo_info['H_tot' ])

        print('Gibbs free energy at 298.15 K')
        print(thermo_info['G_tot' ])

        print('Heat capacity at 298.15 K')
        print(thermo_info['Cv_tot'])
    
    return freq_info, thermo_info

freq_infos, thermo_infos = [], []

for mol in [initial_state_molecule, ts_molecule, fs_molecule]:
    freq_info, thermo_info = get_freq_therm(mol, verbose=True)
    freq_infos.append(freq_info)
    thermo_infos.append(thermo_info)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 3))

length = 1

ax.hlines(
    y=np.array([thermo_infos[0]['G_tot'][0], thermo_infos[1]['G_tot'][0], thermo_infos[2]['G_tot'][0]]),
    xmin=np.arange(3),
    xmax=np.arange(3)+length
)

ax.set_ylabel('G [Hartree]')

plt.show()

In [ ]:
from scipy.constants import Boltzmann, h

hart_to_joule = 4.3597482e-18

def eyring(delta_g, T=298.15):
    return (Boltzmann*T/h)*np.exp(-delta_g/(Boltzmann*T))

# 3
k_is_ts = eyring((thermo_infos[1]['G_tot'][0]-thermo_infos[0]['G_tot'][0])*hart_to_joule)
k_fs_ts = eyring((thermo_infos[1]['G_tot'][0]-thermo_infos[2]['G_tot'][0])*hart_to_joule)

print("Kinetic Rate IS to TS: %e"%k_is_ts)
print("Kinetic Rate FS to TS: %e"%k_fs_ts)

1. Calculate the gibbs free energy change under standard conditions for the SN2 attack of methane with flouride
2. Calculate the gibbs free energy barrier under standard conditions for the SN2 attack of methane with flouride
3. Calculate the kinetic rate of the process under standard conditions
4. Calculate the equilibrium under standard conditions between flouride, hydride, methane, and flouromethane if the initial stoichiometry if 1:1 for flouride and methane